In [1]:
import numpy
from numpy import *
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount("/content/MyDrive")

Mounted at /content/MyDrive


In [3]:
data_raw=genfromtxt("/content/MyDrive/MyDrive/Data/PRNN/Assignment_1/p4_train.csv",delimiter=",")

In [4]:
X_raw,Y_raw=data_raw[1:,1:],data_raw[1:,0]
X_raw=X_raw/255
#X_raw=(X_raw-127.5)/127.5
Y_raw,shape(Y_raw)

(array([0., 1., 2., ..., 7., 8., 9.]), (60000,))

In [5]:
N,d=shape(X_raw)
N,d

(60000, 784)

In [6]:
random_samples=random.choice(arange(N),6000)
Nrs=len(random_samples)
shape(random_samples),Nrs

((6000,), 6000)

In [7]:
X_use,Y_use=X_raw[random_samples],Y_raw[random_samples]

In [11]:
tts_ratio=0.7  #train_test_split_ratio
train_split=int((tts_ratio)*Nrs)
test_split=int(round((1-tts_ratio)*Nrs))
train_split,test_split

(4200, 1800)

In [12]:
X_train,Y_train=X_use[0:train_split,:],Y_use[0:train_split]
X_test,Y_test=X_use[train_split:Nrs,:],Y_use[train_split:Nrs]
shape(X_train),shape(Y_train),shape(X_test),shape(Y_test)

((4200, 784), (4200,), (1800, 784), (1800,))

In [13]:
Ns,d=X_train.shape
Ns,d

(4200, 784)

In [14]:
K=10
labels=arange(K)
labels

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [15]:
Y_onehot=zeros((train_split,len(labels)))

for i in range(train_split):
  for j in (labels):
    if Y_train[i]==j:
      Y_onehot[i,j]=1
Y_train_onehot=Y_onehot

In [16]:
Y_train_onehot[0].shape

(10,)

In [17]:
Y_test

array([7., 4., 2., ..., 4., 2., 5.])

In [18]:
training_data=concatenate((X_train,Y_train_onehot),axis=1)
test_data=concatenate((X_test,Y_test.reshape(len(Y_test),1)),axis=1)

In [20]:
class DenseNeuralNetworks():
  def __init__(self,layers):
    self.layers=layers
    self.L=len(layers)

    self.weights=[]
    for l in range(self.L):
      if l<self.L-1:
        self.weights.append(numpy.random.randn(self.layers[l],self.layers[l+1]))
    self.biases=[]

    for l in range(self.L):
      if l<self.L-1:
        self.biases.append(numpy.random.randn(self.layers[l+1],1))

  def Feedforward(self,x):
    output_activations=[]

    for l in range(self.L):
      if l<self.L-1:
        output_activations.append(zeros((self.layers[l+1],1)))
    for l in range(self.L):
      if l<self.L-1:
        if l==0:

          output_activations[l]=(sigmoid(dot(self.weights[l].transpose(),x.reshape(d,1))+self.biases[l]))
        else:

          output_activations[l]=(sigmoid(dot(self.weights[l].transpose(),output_activations[l-1])+self.biases[l]))

    return output_activations[self.L-2]

  def SGD(self,epochs,mini_batch_size,alpha):

    for ee in range(epochs):
      bb=0


      if (Ns%mini_batch_size==0):

        for mini_batch in  range(int((Ns/mini_batch_size))):
          self.now_use_them_for_updating_Wb(concatenate((X_train[mini_batch+bb:mini_batch+bb+mini_batch_size],Y_train_onehot[mini_batch+bb:mini_batch+bb+mini_batch_size]),axis=1),mini_batch_size,alpha)
          bb=bb+mini_batch_size-1

        print("{0}/{1} in epoch {2}".format(self.now_evaluate_test_data(X_test,Y_test),len(X_test),(ee)))
  def now_use_them_for_updating_Wb(self,training_data,mini_batch_size,alpha):

    dellCw=[]
    dellCb=[]
    sumdellCw=[]
    sumdellCb=[]

    for l in range(self.L):
      if l<self.L-1:
        sumdellCw.append(zeros((self.layers[l],self.layers[l+1])))
        sumdellCb.append(zeros((self.layers[l+1],1)))

    for l in range(self.L):
      if l<self.L-1:
        for ll in range(mini_batch_size):
          dellCw,dellCb=self.Backpropogation(training_data[ll,0:d],training_data[ll,d:])
          sumdellCw[l] = sumdellCw[l]+dellCw[l]
          sumdellCb[l] = sumdellCb[l]+dellCb[l]
    for l in range(self.L):
      if l<self.L-1:
        self.weights[l]=self.weights[l]-(alpha/mini_batch_size)*sumdellCw[l]
        self.biases[l]=self.biases[l]-(alpha/mini_batch_size)*sumdellCb[l]

  def Backpropogation(self,x,y):

    Xi,Yi=x.reshape(d,1),y.reshape(len(labels),1)
    Z=[]
    Activations=[]
    temp_activations1,temp_activations2=0,0


    for l in range(self.L):
      if l<self.L-1:
        if l==0:
          Z.append(dot(self.weights[l].transpose(),Xi)+self.biases[l])
          Activations.append(sigmoid(Z[l]))
          temp_activations1=Activations[l]
        else:
          Z.append(dot(self.weights[l].transpose(),temp_activations1)+self.biases[l])
          temp_activations2=sigmoid(Z[l])
          Activations.append(temp_activations2)
          temp_activations1=temp_activations2

    dellL=multiply((Activations[self.L-2]-Yi),sigmoid(Z[self.L-2])*(1-sigmoid(Z[self.L-2])))

    dells=[]


    for l in range(self.L):
      if l<self.L-1:
        dells.append(zeros((self.layers[l+1],1)))
    for l in range(self.L):
      if l<self.L-1:
        if l==0:
          dells[self.L-2-l]=dellL
          temp_dell1=dells[self.L-2-l]
        else:
          dells[self.L-2-l]=multiply(dot(self.weights[self.L-2-l+1],dells[self.L-2-l+1]),sigmoid(Z[self.L-2-l])*(1-sigmoid(Z[self.L-2-l])))


    dellCw=[]

    for l in range(self.L):
      if l<self.L-1:
        dellCw.append(zeros((self.layers[l],self.layers[l+1])))

    for l in range(self.L):
      if l<self.L-1:
        if l==0:
          dellCw[l]=dot(Xi,dells[l].transpose())
        else:
          dellCw[l]=dot(Activations[l-1],dells[l].transpose())
    dellCb=[]
    for l in range(self.L):
      if l<self.L-1:
        dellCb.append(zeros((self.layers[l+1],1)))
    for l in range(self.L):
      if l<self.L-1:
        dellCb[l]=dells[l]

    return dellCw,dellCb

  def now_evaluate_test_data(self,xtest,ytest):

    y_pred=[]
    for ii in range(len(xtest)):

     y_pred.append(int(argmax(self.Feedforward(X_test[ii]))))

    c=0

    for i in range(len(ytest)):

      if y_pred[i]==int(ytest[i]):
        c=c+1

    return c




def sigmoid(Z):
  return (1/(1+exp(-Z)))

In [25]:
nn=DenseNeuralNetworks([d,16,K])
nn.SGD(50,10,3)

1385/1800 in epoch 0
1532/1800 in epoch 1
1585/1800 in epoch 2
1605/1800 in epoch 3
1615/1800 in epoch 4
1632/1800 in epoch 5
1644/1800 in epoch 6
1650/1800 in epoch 7
1656/1800 in epoch 8
1659/1800 in epoch 9
1661/1800 in epoch 10
1664/1800 in epoch 11
1665/1800 in epoch 12
1667/1800 in epoch 13
1669/1800 in epoch 14
1670/1800 in epoch 15
1672/1800 in epoch 16
1671/1800 in epoch 17
1672/1800 in epoch 18
1673/1800 in epoch 19
1671/1800 in epoch 20
1672/1800 in epoch 21
1672/1800 in epoch 22
1671/1800 in epoch 23
1672/1800 in epoch 24
1672/1800 in epoch 25
1672/1800 in epoch 26
1673/1800 in epoch 27
1669/1800 in epoch 28
1669/1800 in epoch 29
1669/1800 in epoch 30
1669/1800 in epoch 31
1670/1800 in epoch 32
1668/1800 in epoch 33
1667/1800 in epoch 34
1668/1800 in epoch 35
1669/1800 in epoch 36
1668/1800 in epoch 37
1669/1800 in epoch 38
1670/1800 in epoch 39
1671/1800 in epoch 40
1672/1800 in epoch 41
1674/1800 in epoch 42
1675/1800 in epoch 43
1675/1800 in epoch 44
1675/1800 in epoch 4